In [16]:
(<=)

<= (generic function with 58 methods)

In [5]:
1+(1:3)

2:4

In [24]:
a = @operator (j,k,N,M) -> sqrt(j)*π(j∈1:M)*δ(j,k-1)

LoadError: LoadError: UndefVarError: @operator not defined
while loading In[24], in expression starting on line 1

A single (off-)diagonal operator can be decomposed as

$$
A _ {jk} = \tilde{f(j;I)} \delta _ {j,k+s} =  f(j;I) \pi(j;I) \pi(j-s;I)  \delta _ {j,k+s} 
$$

where each projector $\pi(j;I)=\mathbf{1}_{j\in I}$ is one iff $j$ is in the range of valid state labels.
Given any coefficient generating function $f(j)$ 

In [27]:
?inf

search: 

```
inf(f)
```

Returns positive infinity of the floating point type `f` or of the same floating point type as `f`


inf info Inf Inf64 Inf32 Inf16 findfirst @printf maxintfloat getaddrinfo



In [29]:
module Experiments

abstract TAlgebra
abstract LTerm <: TAlgebra


abstract LDiag <: LTerm


type LIdentity
end

type LDGenerated <: LDiag
    fn
end


type LIndicator <: LDiag
    l::Int
    u::Int
end

type LShift <: LDiag
    op::LDiag
    s::Int
end

type LConj <: LDiag
    op::LDiag
end

type LDiagShift <: LTerm
    factors::Vector{LDiag}
    s::Int
end

type TProduct{D} <: TAlgebra
    dfactors::NTuple{D,LDiag}
    ss::NTuple{D,Int}
end


import Base: (.*), (*), conj, transpose, ctranspose

fss(x::LDiag) = [x], 0
fss(x::LTerm) = x.factors, x.s



(.*)(x::LIdentity, y::LIdentity) = x
(.*)(x::LIdentity, y::LTerm) = y
(.*)(x::LTerm, y::LIdentity) = x


conj(x::LIdentity) = x
conj(x::LDiag) = LConj(x)
conj(x::LDiagShift) = LDiagShift([conj(f) for f in x.factors], x.s)

transpose(x::LDiag) = x
transpose(x::LDiagShift) = LDiagShift([LShift(f,-x.s) for f in x.factors], -x.s)
ctranspose(x::LTerm) = transpose(conj(x))


lshift(op::LDiag, s) = s ==0 ? op : LShift(op, s)


function (.*)(x::LTerm, y::LTerm)
    fx, sx = fss(x)
    fy, sy = fss(y)
    fxy = filter(z->~isa(z, LIdentity), [fx; [lshift(op, sx) for op=fy]])
    LDiagShift(fxy, sx+sy)
end


function rcoeffs(T, li::LIndicator, states)
    if li.upper >= li.lower
        return [one(T)*(s∈(li.lower:li.upper)) for s=states]
    else
        return [one(T)*(s>=li.lower) for s=states]
    end
end

function rcoeffs(T, lc::LShift, states)
    rcoeffs(T, lc.op, states-lc.s)
end


function rcoeffs(T, lc::LConj, states)
    conj(rcoeffs(T, lc.op, states))
end

function rcoeffs(T, lc::LShift, states)
    rcoeffs(T, lc.op, states-lc.s)
end


function rcoeffs(T, lg::LDGenerated, states)
    convert(Vector{T}, [lg.fn(s) for s=states])
end


function rcoeffs(T, lds::LDiagShift, states)
    (.*)([rcoeffs(T, d, states) for d=lds.factors]...), lds.s
end

end


create = Experiments.LDiagShift([Experiments.LDGenerated(k->sqrt(k))], 1)
destroy = ctranspose(create)





Experiments.LDiagShift(Experiments.LDiag[Experiments.LShift(Experiments.LConj(Experiments.LDGenerated((anonymous function))),-1)],-1)

In [30]:
Experiments.rcoeffs(Float64, create, 0:20), Experiments.rcoeffs(Float64, destroy, 0:20)

(([0.0,1.0,1.41421,1.73205,2.0,2.23607,2.44949,2.64575,2.82843,3.0  …  3.31662,3.4641,3.60555,3.74166,3.87298,4.0,4.12311,4.24264,4.3589,4.47214],1),([1.0,1.41421,1.73205,2.0,2.23607,2.44949,2.64575,2.82843,3.0,3.16228  …  3.4641,3.60555,3.74166,3.87298,4.0,4.12311,4.24264,4.3589,4.47214,4.58258],-1))

In [31]:
Experiments.rcoeffs(Float64, create.*destroy,0:20)

([0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0  …  11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0],0)

In [35]:
Experiments.rcoeffs(Float64, destroy.*create,0:20)
Experiments.rcoeffs(Float64, create.*create.*destroy.*destroy,0:20)

LoadError: LoadError: DomainError:
sqrt will only return a complex result if called with a complex argument. Try sqrt(complex(x)).
while loading In[35], in expression starting on line 2

In [2]:
comm

LoadError: LoadError: UndefVarError: lds not defined
while loading In[2], in expression starting on line 1